# 情緒分析 - 史丹佛 IMDb 影評資料集

IMDb 資料集共有 50,000 筆 "影評文字", 分為訓練資料與測試資料各 25,000 筆, 每一筆 "影評文字" 都被標記成 "正面評價" 或 "負面評價". 我們希望能建立一個模型, 經過大量 "影評文字" 訓練後, 此模型能用於預測 "影評文字" 是 "正面評價" 或 "負面評價".

[原始出處](http://ai.stanford.edu/~amaas/data/sentiment/)

# 資料集的內容

評論內文
* 正面
> Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, .... Grim, but intriguing, and frightening.
* 負面
> "I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views...

## 1. 觀察資料

In [ ]:
import numpy
from keras.datasets import imdb

# 讀入資料集
(X_train, y_train), (X_test, y_test) = imdb.load_data()
X = numpy.concatenate((X_train, X_test), axis=0)
y = numpy.concatenate((y_train, y_test), axis=0)

print("資料筆數:")
print(X.shape)
print(y.shape)

print("\n分類類別:")
print(numpy.unique(y))

print("\n字詞數量:")
print(len(numpy.unique(numpy.hstack(X))))

print("\n評論平均長度:")
result = [len(x) for x in X]
print("平均 %.2f 字 (%.2f)" % (numpy.mean(result), numpy.std(result)))

## 2 讀入資料集

這裡限制只選「最常被使用」的前 1 萬字, 評論中的詞若不在這個範圍內的，就當不存在。這是文字分析常會做的事。

In [ ]:
top_used_words = #FIX_ME

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_used_words)

### 2-1. 準備輸入的資料

看一下經過「前處理」後的資料，準備輸入模型前的樣子

In [ ]:
x_train[0]

每個數字都代表英文的一個單字。編號方式是資料庫裡所有文字依「出現次數」的排序: 也就是出現頻率越高, 代表的數字就越小。

### 2-2. 模型輸入的處理

因為每篇評論的長度不一，為了訓練方便：

* 設輸入文字長度的上限 (250)
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [ ]:
# 前 10 篇評論的長度
for i in range(10):
    print(len(x_train[i]), end=', ')

In [ ]:
from keras.preprocessing import sequence

# 每篇評論的最大字數
review_length = #FIX_ME

x_train = sequence.pad_sequences(x_train, maxlen=review_length)
x_test = sequence.pad_sequences(x_test, maxlen=review_length)

x_train.shape

In [ ]:
x_train[0]

## 3. 打造你的 RNN

這裡我們選用 LSTM, 基本上用哪種 RNN 寫法都是差不多的!

### 3.1 決定神經網路架構

* 先將 10000 維的文字壓到 256 維
* 然後設定 LSTM 層的輸出維度
* 最後一個 output, 直接用 sigmoid 送出

### 3.2 Word Embedding

文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少 (10000), 還有我們打算壓到幾維 (256)。

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

# 文字向量的維度
vector_dim = #FIX_ME

# LSTM 層輸出的維度
num_lstm_cell = #FIX_ME

model = Sequential()
model.add(Embedding(10000, vector_dim))
model.add(LSTM(num_lstm_cell))

# 最後一層 dense output, 直接用 sigmoid 送出
model.add(Dense(1, activation='sigmoid'))

# 模型目前的架構
model.summary()

### 3.3. 損失函數

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

## 4. 模型訓練

In [ ]:
model.fit(x_train, y_train, 
          batch_size=32, 
          epochs=1)

## 5. 結果

### 5.1 分數

來看看測試資料的預測分數。

In [ ]:
score = model.evaluate(x_test, y_test)

In [ ]:
print('測試資料的 loss:', score[0])
print('測試資料正確率:', score[1])

### 5.2 儲存結果

把模型的學習結果存檔。

In [ ]:
model_name = 'your_model_name.h5'#FIX_ME

model_json = model.to_json()
open('imdb_model_architecture.json', 
     'w').write(model_json)
model.save_weights(model_name)

## 參考資料

1. [深度學習 MOOC 檔案](https://github.com/yenlung/Deep-Learning-MOOC)

2. [deep-learning-with-keras-notebooks](http://nbviewer.jupyter.org/github/erhwenkuo/deep-learning-with-keras-notebooks/blob/master/1.a-rnn-introduction.ipynb)

3. [Predict Sentiment From Movie Reviews Using Deep Learning](https://machinelearningmastery.com/predict-sentiment-movie-reviews-using-deep-learning/)
4. [程式扎記 Keras - IMDb 網路電影資料集](http://puremonkey2010.blogspot.tw/2017/09/toolkit-keras-imdb.html)
5. [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/)